#### Author: Thejasvi Beleyur
#### Last updated: 29-02-2020

### Annotations only between 02:00-02:10 because of camera movement. 
This notebook will detail all the steps required to get matching audio snippets for the annotations of bat flights made for 2018-08-19 between 02:00-02:10 hours. Only 10 minutes are being annotated in the hour because cam01 moved - meaning that we couldn't maintain the two-camera annotation protocol for this hour. 

This notebook is basically a copy of 'Matching annotations to audio 2018-08-16 2150-2300.ipynb'. 

### Steps carried out before hand:
1. Generating the video sync for the video file *OrlovaChukaDome_01_20180819_02.00.00-03.00.00[R]....avi* with frame-wise light intensities and time stamps

2. Video annotations of the horseshoebats flying done by  Aditya Kanjoor. 


In [3]:
import matplotlib.pyplot as plt
import pandas as pd 
from tqdm import tqdm

package_folder = '../../../bin/'
import os
import sys 
sys.path.append(package_folder) # include the modules in the outer folder in the search path
from process_video_annotations import video_sync_over_annotation_block



In [4]:
%matplotlib notebook

In [5]:
# loading the annotations
annotations = pd.read_csv('annotations/corrected_HBC_video_annotations_Aditya/2018-08-19_23_Aditya_corrected.csv')

In [6]:
annotations.head()

video_path           annotation_id  \
0  OrlovaChukaDome_02_20180819_02.00.00-03.00.00[...  Aditya_2018-08-19_23_1   
1  OrlovaChukaDome_02_20180819_02.00.00-03.00.00[...  Aditya_2018-08-19_23_2   
2  OrlovaChukaDome_02_20180819_02.00.00-03.00.00[...  Aditya_2018-08-19_23_3   
3  OrlovaChukaDome_02_20180819_02.00.00-03.00.00[...  Aditya_2018-08-19_23_4   
4  OrlovaChukaDome_02_20180819_02.00.00-03.00.00[...  Aditya_2018-08-19_23_5   

   Sampling minute  timestamp from camera      start_timestamp  \
0                0                      2  2018-08-19 02:00:11   
1                1                      2  2018-08-19 02:01:06   
2                1                      2  2018-08-19 02:01:21   
3                1                      2  2018-08-19 02:01:22   
4                1                      2  2018-08-19 02:01:23   

   start_framenumber        end_timestamp  end_framenumber  \
0                 20  2018-08-19 02:00:12               15   
1                  6  2018-08-19 02:01:06               17   
2                 17  2018-08-19 02:01:22                6   
3                 13  2018-08-19 02:01:22               19   
4                  4  2018-08-19 02:01:23                9   

   no. of flying bats bat behaviour  strength (1 - strong, 0-weak)  \
0                   1           NaN                              0   
1                   1           NaN                              1   
2                   1           NaN                              1   
3                   2             T                              1   
4                   2             P                              0   

  weak in (Behaviour Be, Number Bn)  \
0                               NaN   
1                               NaN   
2                               NaN   
3                               NaN   
4                               NaN   

                                            Comments  Details on the comment  \
0  The bat has exited only out of the camera view...                     NaN   
1                                                NaN                     NaN   
2                                                NaN                     NaN   
3                                                NaN                     NaN   
4  Lasting only for a few frames after the buffer...                     NaN   

  annotator  
0    Aditya  
1    Aditya  
2    Aditya  
3    Aditya  
4    Aditya

In [7]:
annotations.shape

(44, 15)

In [8]:
annots_w_bats = annotations[annotations['no. of flying bats']>0].reset_index(drop=True)

In [9]:
annotations.shape

(44, 15)

In [10]:
annots_w_bats.shape

(44, 15)

In [11]:
annots_w_bats.head()

video_path           annotation_id  \
0  OrlovaChukaDome_02_20180819_02.00.00-03.00.00[...  Aditya_2018-08-19_23_1   
1  OrlovaChukaDome_02_20180819_02.00.00-03.00.00[...  Aditya_2018-08-19_23_2   
2  OrlovaChukaDome_02_20180819_02.00.00-03.00.00[...  Aditya_2018-08-19_23_3   
3  OrlovaChukaDome_02_20180819_02.00.00-03.00.00[...  Aditya_2018-08-19_23_4   
4  OrlovaChukaDome_02_20180819_02.00.00-03.00.00[...  Aditya_2018-08-19_23_5   

   Sampling minute  timestamp from camera      start_timestamp  \
0                0                      2  2018-08-19 02:00:11   
1                1                      2  2018-08-19 02:01:06   
2                1                      2  2018-08-19 02:01:21   
3                1                      2  2018-08-19 02:01:22   
4                1                      2  2018-08-19 02:01:23   

   start_framenumber        end_timestamp  end_framenumber  \
0                 20  2018-08-19 02:00:12               15   
1                  6  2018-08-19 02:01:06               17   
2                 17  2018-08-19 02:01:22                6   
3                 13  2018-08-19 02:01:22               19   
4                  4  2018-08-19 02:01:23                9   

   no. of flying bats bat behaviour  strength (1 - strong, 0-weak)  \
0                   1           NaN                              0   
1                   1           NaN                              1   
2                   1           NaN                              1   
3                   2             T                              1   
4                   2             P                              0   

  weak in (Behaviour Be, Number Bn)  \
0                               NaN   
1                               NaN   
2                               NaN   
3                               NaN   
4                               NaN   

                                            Comments  Details on the comment  \
0  The bat has exited only out of the camera view...                     NaN   
1                                                NaN                     NaN   
2                                                NaN                     NaN   
3                                                NaN                     NaN   
4  Lasting only for a few frames after the buffer...                     NaN   

  annotator  
0    Aditya  
1    Aditya  
2    Aditya  
3    Aditya  
4    Aditya

### Carrying out the common-fps conversion:


In [12]:
# load the video sync file with corrected timestamps
video_sync_file = 'videosync_OrlovaChukaDome_01_20180819_02.00.00-03.00.00[R][@3514][1].aviw_suggestions_[timestamp_corrected]_[signal_checked].csv'
path_to_videosync = os.path.join('raw_sync',
                                 video_sync_file)
video_sync = pd.read_csv(path_to_videosync)

In [13]:
video_sync.head()

Unnamed: 0  frame_number  led_intensity            timestamp  \
0           0             1       0.722407  2018-08-19 02:00:00   
1           1             2       0.756906  2018-08-19 02:00:00   
2           2             3       0.756906  2018-08-19 02:00:00   
3           3             4       0.756906  2018-08-19 02:00:00   
4           4             5       0.756906  2018-08-19 02:00:00   

    timestamp_verified user_suggestion  
0  2018-08-19 02:00:00       maybeokay  
1  2018-08-19 02:00:00       maybeokay  
2  2018-08-19 02:00:00       maybeokay  
3  2018-08-19 02:00:00       maybeokay  
4  2018-08-19 02:00:00       maybeokay

In [14]:
kwargs = {'timestamp_pattern': '%Y-%m-%d %H:%M:%S'}
kwargs['min_fps']= 21 # Hz
kwargs['min_durn'] = 45.0 # seconds 
kwargs['common_fps'] = 25 # Hz
kwargs['output_folder'] = './common_fps/Aditya_annotations/2018-08-18/'

unprocessed_annotations = []

for i, each_annotation in tqdm(annots_w_bats.iterrows()):
    try:
        success = video_sync_over_annotation_block(each_annotation,video_sync, 
                                                     **kwargs)
    except :
        unprocessed_annotations.append(i)
        print('Unable to process annotation: %s'%annots_w_bats['annotation_id'][i])


0it [00:00, ?it/s]../../../bin/process_video_annotations.py:389: UserWarning: Some timestamps below min fps!
  warnings.warn('Some timestamps below min fps!')
../../../bin/process_video_annotations.py:310: UserWarning: FPS fell below the minimum required - ignoring this annotation
  warnings.warn('FPS fell below the minimum required - ignoring this annotation')
../../../bin/process_video_annotations.py:115: UserWarning: could not bring above annotation to common fps,                      please check a) if the corresponding audio file is present                        b) if the ON/OFF of the light signal is clean                           
  ')
1it [00:04,  4.66s/it]

('timestamps below min fps: ', array(['2018-08-19 02:00:53'], dtype=object))
video_path                           OrlovaChukaDome_02_20180819_02.00.00-03.00.00[...
annotation_id                                                   Aditya_2018-08-19_23_1
Sampling minute                                                                      0
timestamp from camera                                                                2
start_timestamp                                                    2018-08-19 02:00:11
start_framenumber                                                                   20
end_timestamp                                                      2018-08-19 02:00:12
end_framenumber                                                                     15
no. of flying bats                                                                   1
bat behaviour                                                                      NaN
strength (1 - strong, 0-weak)                        

/home/tbeleyur/anaconda2/envs/horseshoebat/lib/python2.7/site-packages/scipy/signal/signaltools.py:2223: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  Y[sl] = X[sl]
/home/tbeleyur/anaconda2/envs/horseshoebat/lib/python2.7/site-packages/scipy/signal/signaltools.py:2225: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  Y[sl] = X[sl]
/home/tbeleyur/anaconda2/envs/horseshoebat/lib/python2.7/site-packages/scipy/signal/signaltools.py:2233: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` 

Saved annotation :Aditya_2018-08-19_23_2


3it [00:18,  5.94s/it]

...

('timestamps below min fps: ', array(['2018-08-19 02:08:39'], dtype=object))
video_path                           OrlovaChukaDome_02_20180819_02.00.00-03.00.00[...
annotation_id                                                  Aditya_2018-08-19_23_42
Sampling minute                                                                      8
timestamp from camera                                                                2
start_timestamp                                                    2018-08-19 02:08:20
start_framenumber                                                                   15
end_timestamp                                                      2018-08-19 02:08:22
end_framenumber                                                                     24
no. of flying bats                                                                   1
bat behaviour                                                                      NaN
strength (1 - strong, 0-weak)                        

43it [04:35,  6.42s/it]

('timestamps below min fps: ', array(['2018-08-19 02:08:39'], dtype=object))
video_path                           OrlovaChukaDome_02_20180819_02.00.00-03.00.00[...
annotation_id                                                  Aditya_2018-08-19_23_43
Sampling minute                                                                      8
timestamp from camera                                                                2
start_timestamp                                                    2018-08-19 02:08:24
start_framenumber                                                                   12
end_timestamp                                                      2018-08-19 02:08:27
end_framenumber                                                                      2
no. of flying bats                                                                   1
bat behaviour                                                                      NaN
strength (1 - strong, 0-weak)                        

44it [04:43,  6.95s/it]

Saved annotation :Aditya_2018-08-19_23_44


### Matching Audio from the common_fps files

In [15]:
### Finding the matching audio snippet !! 
import glob 
import soundfile as sf
from audio_for_videoannotation import match_video_sync_to_audio, search_for_best_fit

#all_commonfps = glob.glob('common_fps_video_sync*') # get all the relevant common_fps_sync files
all_commonfps = glob.glob('common_fps/Aditya_annotations/2018-08-18/common_fps_video_syncAditya_2018-08-19_23*')
audio_folder = '/home/tbeleyur/Documents/horseshoe_bat/audio_non_spikey/2018-08-18/' # the current folder

# the output folders
audiosync_folder = './sync_audio/'
audioannotation_folder = './annotation_audio/'
fs = 250000 # change according to the recording sampling rate in Hz!! 


In [16]:
all_ccs = []
files_to_run = sorted(all_commonfps)

def generate_matching_audio(each_commonfps, audio_folder, 
                           **kwargs):
    '''
    '''
    print(each_commonfps)
    try:
        video_sync = pd.read_csv(each_commonfps)
        best_audio, syncblock_audio, crosscoef = match_video_sync_to_audio(video_sync, audio_folder, 
                                                                           **kwargs)
        all_ccs.append(crosscoef)
        fname  = os.path.split(each_commonfps)[-1]
        annotation_id = '-'.join(os.path.split(fname)[-1].split('common_fps_video_sync')[-1:])[:-4]

        audiosync_folder = kwargs.get('audiosync_folder', 'sync_audio')
        audioannotation_folder = kwargs.get('audioannotation_folder', 'annotation_audio')

    
        audio_file_base_name = annotation_id+'.WAV'
        sf.write(audiosync_folder+'matching_sync_'+audio_file_base_name, syncblock_audio,fs)
        sf.write(audioannotation_folder+'matching_annotaudio_'+audio_file_base_name, best_audio,fs)
        return(True)
    except:
        print('Could not save ', each_commonfps)
        return(False)



In [17]:
glob.glob(audio_folder+'*.WAV')

['/home/tbeleyur/Documents/horseshoe_bat/audio_non_spikey/2018-08-18/non_spikey_T0000822.WAV',
 '/home/tbeleyur/Documents/horseshoe_bat/audio_non_spikey/2018-08-18/non_spikey_T0000974.WAV',
 '/home/tbeleyur/Documents/horseshoe_bat/audio_non_spikey/2018-08-18/non_spikey_T0000923.WAV',
 '/home/tbeleyur/Documents/horseshoe_bat/audio_non_spikey/2018-08-18/non_spikey_T0000888.WAV',
 '/home/tbeleyur/Documents/horseshoe_bat/audio_non_spikey/2018-08-18/non_spikey_T0000851.WAV',
 '/home/tbeleyur/Documents/horseshoe_bat/audio_non_spikey/2018-08-18/non_spikey_T0000918.WAV',
 '/home/tbeleyur/Documents/horseshoe_bat/audio_non_spikey/2018-08-18/non_spikey_T0000858.WAV',
 '/home/tbeleyur/Documents/horseshoe_bat/audio_non_spikey/2018-08-18/non_spikey_T0000919.WAV',
 '/home/tbeleyur/Documents/horseshoe_bat/audio_non_spikey/2018-08-18/non_spikey_T0000984.WAV',
 '/home/tbeleyur/Documents/horseshoe_bat/audio_non_spikey/2018-08-18/non_spikey_T0000963.WAV',
 '/home/tbeleyur/Documents/horseshoe_bat/audio_non

In [18]:
os.listdir(audio_folder)

['non_spikey_T0000822.WAV',
 'non_spikey_T0000974.WAV',
 'non_spikey_T0000923.WAV',
 'non_spikey_T0000888.WAV',
 'non_spikey_T0000851.WAV',
 'non_spikey_T0000918.WAV',
 'non_spikey_T0000858.WAV',
 'non_spikey_T0000919.WAV',
 'non_spikey_T0000984.WAV',
 'non_spikey_T0000963.WAV',
 'non_spikey_T0000902.WAV',
 'non_spikey_T0000911.WAV',
 'non_spikey_T0000959.WAV',
 'non_spikey_T0000969.WAV',
 'non_spikey_T0000869.WAV',
 'non_spikey_T0000886.WAV',
 'non_spikey_T0000949.WAV',
 'non_spikey_T0000912.WAV',
 'non_spikey_T0000947.WAV',
 'non_spikey_T0000986.WAV',
 'non_spikey_T0000903.WAV',
 'non_spikey_T0000962.WAV',
 'non_spikey_T0000952.WAV',
 'non_spikey_T0000882.WAV',
 'non_spikey_T0000839.WAV',
 'non_spikey_T0000917.WAV',
 'non_spikey_T0000880.WAV',
 'non_spikey_T0000823.WAV',
 'non_spikey_T0000931.WAV',
 'non_spikey_T0000916.WAV',
 'non_spikey_T0000881.WAV',
 'non_spikey_T0000929.WAV',
 'non_spikey_T0000956.WAV',
 'non_spikey_T0000857.WAV',
 'non_spikey_T0000848.WAV',
 'non_spikey_T000095

In [19]:
all_commonfps

['common_fps/Aditya_annotations/2018-08-18/common_fps_video_syncAditya_2018-08-19_23_34.csv',
 'common_fps/Aditya_annotations/2018-08-18/common_fps_video_syncAditya_2018-08-19_23_17.csv',
 'common_fps/Aditya_annotations/2018-08-18/common_fps_video_syncAditya_2018-08-19_23_3.csv',
 'common_fps/Aditya_annotations/2018-08-18/common_fps_video_syncAditya_2018-08-19_23_12.csv',
 'common_fps/Aditya_annotations/2018-08-18/common_fps_video_syncAditya_2018-08-19_23_38.csv',
 'common_fps/Aditya_annotations/2018-08-18/common_fps_video_syncAditya_2018-08-19_23_6.csv',
 'common_fps/Aditya_annotations/2018-08-18/common_fps_video_syncAditya_2018-08-19_23_4.csv',
 'common_fps/Aditya_annotations/2018-08-18/common_fps_video_syncAditya_2018-08-19_23_2.csv',
 'common_fps/Aditya_annotations/2018-08-18/common_fps_video_syncAditya_2018-08-19_23_16.csv',
 'common_fps/Aditya_annotations/2018-08-18/common_fps_video_syncAditya_2018-08-19_23_33.csv',
 'common_fps/Aditya_annotations/2018-08-18/common_fps_video_sync

In [20]:
audio_folder

'/home/tbeleyur/Documents/horseshoe_bat/audio_non_spikey/2018-08-18/'

In [21]:
len(all_commonfps)

24

In [22]:
len(all_commonfps)

24

In [23]:
for each in sorted(all_commonfps):
    generate_matching_audio(each, audio_folder, audio_sync_spikey=False, file_subset=('non_spikey_T0000855',
                                                                                      'non_spikey_T0000870'),
                            audio_fileformat='*.WAV',
                          audiosync_folder='sync_audio/Aditya_annotations/',
                           audioannotation_folder='annotation_audio/Aditya_annotations/')


common_fps/Aditya_annotations/2018-08-18/common_fps_video_syncAditya_2018-08-19_23_12.csv
('video_fps obtained is :', 25)
Did not find user-provided sample rate - getting it from first file that matches format!
('sampling rate is : ', 250000)
.....finding best audio segment.....


  0%|          | 0/15 [00:00<?, ?it/s]

('max mins for upsampled video', -0.9105283243143308, 1.0)
('max mins', -1.0, 0.9247948730428763)


/home/tbeleyur/anaconda2/envs/horseshoebat/lib/python2.7/site-packages/scipy/signal/signaltools.py:491: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return x[reverse].conj()
  7%|▋         | 1/15 [00:06<01:24,  6.07s/it]

('max mins', -1.0, 0.9609768360886519)


...

('max mins', -0.9530957512155002, 1.0)


 93%|█████████▎| 14/15 [01:04<00:04,  4.54s/it]

('max mins', -0.9789960168861033, 1.0)


100%|██████████| 15/15 [01:09<00:00,  4.64s/it]


('Best file pairs are: ', 'non_spikey_T0000856.WAV*non_spikey_T0000857.WAV')
Start and end indices around peak:                  start index: 11538474		          end index: 23038474                  total samples:30015488
('AV Sync was above threshold: ', 0.7174008031792526)


### Taking stock of the AV sync cross-correlation coefficients:
Most of the matches are $\geq0.8$ , but there are some that are start between 02:01:21 to 02:01:23. These annotations have a lower cross-corr coefficient than other (~0.7), and are characterised by a good match in the sync_audio, except for hte last few (6-8 seconds) seconds. I'm guessing this might be because of a weird framerate anomaly that's showing up in the common_fps files. 

The ~0.7 cc coefficient matched annotations are ```2018-08-19_23_3, 2018-08-19_23_4, 2018-08-19_23_5, 2018-08-19_23_6```. The 

### Deleting one bad match: 2018-08-19_23_44
The sync and annotation audio for Annotation id 44 will be deleted because there was actually *no* video signal to actually match. This annotation fell in the period in the video_sync file that was zero-filled. 

In [26]:
import numpy as np 

In [37]:
start_frame = int(np.argwhere(video_sync['timestamp_verified']=='2018-08-19 02:01:21')[0])
end_frame = int(np.argwhere(video_sync['timestamp_verified']=='2018-08-19 02:02:08')[-1])

start_frame, end_frame
relevant_rows = video_sync.iloc[start_frame:end_frame+1,:]
timestamps, counts = np.unique(relevant_rows['timestamp_verified'], return_counts=True)
pd.DataFrame(data={'timestamp':timestamps, 'num_frames':counts})

num_frames            timestamp
0           22  2018-08-19 02:01:21
1           23  2018-08-19 02:01:22
2           21  2018-08-19 02:01:23
3           22  2018-08-19 02:01:24
4           23  2018-08-19 02:01:25
5           21  2018-08-19 02:01:26
6           22  2018-08-19 02:01:27
7           23  2018-08-19 02:01:28
8           21  2018-08-19 02:01:29
9           22  2018-08-19 02:01:30
10          22  2018-08-19 02:01:31
11          22  2018-08-19 02:01:32
12          22  2018-08-19 02:01:33
13          22  2018-08-19 02:01:34
14          22  2018-08-19 02:01:35
15          22  2018-08-19 02:01:36
16          22  2018-08-19 02:01:37
17          22  2018-08-19 02:01:38
18          22  2018-08-19 02:01:39
19          22  2018-08-19 02:01:40
20          22  2018-08-19 02:01:41
21          22  2018-08-19 02:01:42
22          22  2018-08-19 02:01:43
23          22  2018-08-19 02:01:44
24          22  2018-08-19 02:01:45
25          22  2018-08-19 02:01:46
26          22  2018-08-19 02:01:47
27          22  2018-08-19 02:01:48
28          22  2018-08-19 02:01:49
29          22  2018-08-19 02:01:50
30          22  2018-08-19 02:01:51
31          22  2018-08-19 02:01:52
32          22  2018-08-19 02:01:53
33          22  2018-08-19 02:01:54
34          22  2018-08-19 02:01:55
35          22  2018-08-19 02:01:56
36          22  2018-08-19 02:01:57
37          23  2018-08-19 02:01:58
38          21  2018-08-19 02:01:59
39          44  2018-08-19 02:02:00
40          22  2018-08-19 02:02:01
41          22  2018-08-19 02:02:02
42          22  2018-08-19 02:02:03
43          22  2018-08-19 02:02:04
44          23  2018-08-19 02:02:05
45          21  2018-08-19 02:02:06
46          22  2018-08-19 02:02:07
47          22  2018-08-19 02:02:08

### The problem timestamp : 
```2018-08-19 02:02:00``` appears 44 times - and thus messes up with the cross correlation coefficients! But otherwise, the matches are correct!